In [ ]:
#==================================================================
# CNN サンプル (課題1)
#==================================================================
import os

from keras           import optimizers
from keras.callbacks import History
from keras.datasets  import mnist
from keras.models    import Sequential, Model
from keras.layers    import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout, Input
from keras.utils     import to_categorical

from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16  import VGG16

import matplotlib.pyplot as plt


#--------------------------------------------------------
# グローバル定数
#--------------------------------------------------------
# モデル関連
G_INPUT_IMAGE_WIDTH   = 32                 # 入力画像幅
G_INPUT_IMAGE_HEIGHT  = 32                 # 入力画像高さ
G_INPUT_IMAGE_CH      = 3                   # 深度

G_OUTPUT_FC_DENCE     = 3                   # 分類数(全結合層出力)
G_OUTPUT_ACTIVATION   = 'softmax'           # 全結合層の活性化関数

# データ関連
G_DIR_TRAIN_DATA      = '../exercise1/training'     # 学習用データ
G_DIR_VALIDATION_DATA = '../exercise1/test'         # 検証用データ
G_DIR_RESULT          = 'result'                    # 結果出力フォルダ

# 訓練関連
G_BATCH_SIZE          = 32                  # バッチサイズ
G_EPOCHS              = 5                   # エポック数


#--------------------------------------------------------
# 学習結果をグラフ表示する(ACC, VAL_ACC)
# 
# param: history     generator.fitの戻り値(History)
#--------------------------------------------------------
def plot_result(history:History):
    plt.ylim(0.0, 1)
    plt.plot(history.history['acc'], label="acc")
    plt.plot(history.history['val_acc'], label="val_acc")
    plt.plot(history.history['loss'], label="loss")
    plt.plot(history.history['val_loss'], label="val_loss")
    plt.legend()
    plt.show()


#--------------------------------------------------------
# トレーニング及びバリデーション用のセットを生成するジェネレータを作成
# 
# return: train_gen         学習用ジェネレータ
# return: validation_gen    検証用ジェネレータ
#--------------------------------------------------------
def create_generator():

    # 訓練用データのジェネレータ
    train_datagen = ImageDataGenerator(
        rescale=1.0 / 255,        # 0-255 -> 0 -> 1.0
        shear_range=0.2,          # シアー(斜め)
        zoom_range=0.2,           # 拡縮
        horizontal_flip=True,     # 垂直軸に対するフリップ
        rotation_range=10)        # ランダムに回転
    
    #　検証用データのジェネレータ
    #　検証用データにはデータ拡張は行わない
    validation_datagen = ImageDataGenerator(
        rescale=1.0 / 255
    )

    # 訓練用セットを生成するジェネレータを作成
    train_generator = train_datagen.flow_from_directory(
        G_DIR_TRAIN_DATA,                                        #入力フォルダ
        target_size=(G_INPUT_IMAGE_WIDTH, G_INPUT_IMAGE_HEIGHT), #画像の自動リサイズ指定
        batch_size=G_BATCH_SIZE,                                 #ミニバッチ学習サイズ
        class_mode="categorical",                                # one-hot
        shuffle=True)

    # 検証用セットを生成するジェネレータを作成
    validation_generator = validation_datagen.flow_from_directory(
        G_DIR_VALIDATION_DATA,
        target_size=(G_INPUT_IMAGE_WIDTH, G_INPUT_IMAGE_HEIGHT),
        batch_size=G_BATCH_SIZE,
        class_mode="categorical",
        shuffle=True)

    return train_generator, validation_generator

#--------------------------------------------------------
# CNNモデルの生成(簡易版オリジナルモデル)
# 
# return: model
#--------------------------------------------------------
def create_cnn_model():
    model = Sequential()

    # Conv：フィルタ数32  カーネルサイズ3x3　活性化関数relu
    # Pool:プールサイズ2x2
    model.add(Conv2D(32, 3, input_shape=(G_INPUT_IMAGE_WIDTH, G_INPUT_IMAGE_HEIGHT, G_INPUT_IMAGE_CH)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Conv:フィルタ数32  カーネルサイズ3x3　活性化関数relu
    # Pool:プールサイズ2x2
    model.add(Conv2D(32, 3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Conv:フィルタ数32  カーネルサイズ3x3　活性化関数relu
    # Pool:プールサイズ2x2
    model.add(Conv2D(64, 3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2))) 

    # 平滑化層
    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    # 全結合層 ノード数3　活性化関数 softmax
    model.add(Dense(G_OUTPUT_FC_DENCE))
    model.add(Activation(G_OUTPUT_ACTIVATION))
    
    # 要約出力
    model.summary()

    # 損失関数
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

#--------------------------------------------------------
# VGG16の全結合層以外を再利用しFine-tuningするためのモデルを生成
#
# return: model
#--------------------------------------------------------
def create_cnn_model_based_vgg16():

    # VGG16モデルとImageNet学習済み重みをロード
    # 全結合層はカスタマイズするためinclude_top=False
    input_tensor = Input(shape=(G_INPUT_IMAGE_WIDTH, G_INPUT_IMAGE_HEIGHT, G_INPUT_IMAGE_CH))
    vgg16_model  = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)

    # 全結合層を構築
    # Flattenへの入力指定はバッチ数を除く
    top_model = Sequential()
    top_model.add(Flatten(input_shape=vgg16_model.output_shape[1:]))
    top_model.add(Dense(256, activation='relu'))
    top_model.add(Dropout(0.5))
    top_model.add(Dense(G_OUTPUT_FC_DENCE, activation=G_OUTPUT_ACTIVATION))


    # vgg16_model => keras.engine.training.Model
    # top_model   => Sequential
    # Functional APIでモデルを結合
    model = Model(inputs=vgg16_model.input, outputs=top_model(vgg16_model.output))

    # 最後のconv層の直前までの層を学習対象からはずす
    for layer in model.layers[:15]:
        layer.trainable = False

    # 要約出力
    # Total params: 16,812,353
    # Trainable params: 9,177,089
    # Non-trainable params: 7,635,264
    model.summary()

    # Optimizer:SGD (学習率を小さ目に設定)
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.SGD(lr=0.0001, momentum=0.9),
                  metrics=['accuracy'])

    return model

#---------------------------------------------------------
# モデル訓練
#---------------------------------------------------------
def training(model:Model, train_gen:ImageDataGenerator, validation_gen:ImageDataGenerator):

    history = model.fit_generator(
        train_gen,                                             # 学習用generetor
        steps_per_epoch=train_gen.samples/G_BATCH_SIZE,        # 次のエポックの開始前までにgeneratorから生成されるサンプル数
        epochs=G_EPOCHS,                                       # エポック数
        validation_data=validation_gen,                        # 検証用generator 
        validation_steps=validation_gen.samples/G_BATCH_SIZE)  # 各エポックの終わりに検証用ジェネレータから使用するステップ数

    return history

#---------------------------------------------------------
# モデル圧縮
#---------------------------------------------------------
def distillation(teacher_model:Model,train_gen:ImageDataGenerator):
	for i in range(train_gen.samples):
        batch = g.next()

	return student_model



# --------------------------------------------------------
# main関数
# --------------------------------------------------------
def main():
    try:
        model_gen = [create_cnn_model,create_cnn_model_based_vgg16]
        
        print("Create generetors...")
        train_gen, validation_gen = create_generator()
        
        for i in range(len(model_gen)):
            print("Create model...")
            model = model_gen[i]() 
            print("Start training...")
            history = training(model, train_gen, validation_gen)
            plot_result(history)
            print("Save weights...")
            model.save_weights(os.path.join(G_DIR_RESULT, 'history_' + str(i) + 'h5'))

    except Exception as e:
        print("EXCEPTION:", e)
    finally:
        print("RESULT")

#--------------------------------------------------------
# トップレベルスクリプト実行ポイント
#--------------------------------------------------------
if __name__ == "__main__":
    main()